In [1]:
import pandas as pd

In [53]:
data = pd.read_csv('outputs/data_frame.csv')
data.head()

,movement_type,hexagon_origin,lat_origin,lng_origin,hexagon_destination,lat_destination,lng_destination,start_at,end_at,duration,revenue,cost,start_time,end_time
0,empty_move,871e80420ffffff,12.543854,41.808747,871e80523ffffff,12.443846,41.919537,480.000000,499.298422,19.298422,0.000000,13.508895,2025-05-31 08:00:09.074209,2025-05-31 08:19:09.085166
1,ride,871e80523ffffff,12.443846,41.919537,871e80422ffffff,12.529863,41.788283,499.298422,566.648474,67.350052,29.415006,14.773140,2025-05-31 08:19:09.074279,2025-05-31 09:27:09.085177
2,empty_move,871e80422ffffff,12.529863,41.788283,871e80518ffffff,12.465912,41.823248,566.648474,575.205269,8.556795,0.000000,5.989757,2025-05-31 09:27:09.074281,2025-05-31 09:35:09.085178
3,ride,871e80518ffffff,12.465912,41.823248,871e805a0ffffff,12.329468,41.794715,575.205269,628.367112,53.161843,23.446401,10.713259,2025-05-31 09:35:09.074283,2025-05-31 10:28:09.085179
4,empty_move,871e805a0ffffff,12.329468,41.794715,871e80572ffffff,12.569215,41.888591,628.367112,657.587371,29.220258,0.000000,20.454181,2025-05-31 10:28:09.074284,2025-05-31 10:58:09.085180


In [15]:
from datetime import datetime
def change_time(time):
    today = datetime.now()
    hours = int(time // 60)
    minutes = int(round(time % 60))
    return today.replace(hour=hours, minute=minutes)

data['start_time'] = data['start_at'].apply(change_time)
data['end_time'] = data['end_at'].apply(change_time)


In [16]:
data

,movement_type,hexagon_origin,lat_origin,lng_origin,hexagon_destination,lat_destination,lng_destination,start_at,end_at,duration,revenue,cost,start_time,end_time
0,empty_move,871e80420ffffff,12.543854,41.808747,871e80523ffffff,12.443846,41.919537,480.000000,499.298422,19.298422,0.000000,13.508895,2025-05-31 08:00:09.074209,2025-05-31 08:19:09.085166
1,ride,871e80523ffffff,12.443846,41.919537,871e80422ffffff,12.529863,41.788283,499.298422,566.648474,67.350052,29.415006,14.773140,2025-05-31 08:19:09.074279,2025-05-31 09:27:09.085177
2,empty_move,871e80422ffffff,12.529863,41.788283,871e80518ffffff,12.465912,41.823248,566.648474,575.205269,8.556795,0.000000,5.989757,2025-05-31 09:27:09.074281,2025-05-31 09:35:09.085178
3,ride,871e80518ffffff,12.465912,41.823248,871e805a0ffffff,12.329468,41.794715,575.205269,628.367112,53.161843,23.446401,10.713259,2025-05-31 09:35:09.074283,2025-05-31 10:28:09.085179
4,empty_move,871e805a0ffffff,12.329468,41.794715,871e80572ffffff,12.569215,41.888591,628.367112,657.587371,29.220258,0.000000,20.454181,2025-05-31 10:28:09.074284,2025-05-31 10:58:09.085180
5,wait,871e80572ffffff,12.569215,41.888591,871e80572ffffff,12.569215,41.888591,657.587371,889.997371,232.410000,0.000000,0.000000,2025-05-31 10:58:09.074285,2025-05-31 14:50:09.085181
6,ride,871e80572ffffff,12.569215,41.888591,871e86259ffffff,12.282180,41.811118,890.000000,975.197927,85.197927,52.429195,23.061701,2025-05-31 14:50:09.074286,2025-05-31 16:15:09.085182
7,empty_move,871e86259ffffff,12.282180,41.811118,871e8625cffffff,12.234883,41.827496,975.197927,980.827253,5.629326,0.000000,3.940528,2025-05-31 16:15:09.074287,2025-05-31 16:21:09.085183
8,wait,871e8625cffffff,12.234883,41.827496,871e8625cffffff,12.234883,41.827496,980.827253,1005.997253,25.170000,0.000000,0.000000,2025-05-31 16:21:09.074288,2025-05-31 16:46:09.085184
9,ride,871e8625cffffff,12.234883,41.827496,871e8051dffffff,12.479891,41.843712,1006.000000,1075.607298,69.607298,32.288130,18.592008,2025-05-31 16:46:09.074289,2025-05-31 17:56:09.085185


In [59]:
import pandas as pd
import json
from datetime import datetime, timedelta
import h3

# Load your dataset
df = data
features = []

for i, row in df.iterrows():
    if row['movement_type'] in ['ride', 'empty_move', 'wait']:
        start_time = pd.to_datetime(row['start_time'])
        end_time = pd.to_datetime(row['end_time'])
        duration_sec = int((end_time - start_time).total_seconds())
        
        # Get origin and destination coordinates (corrected)
        origin_lat = row['lng_origin']
        origin_lng = row['lat_origin']
        dest_lat = row['lng_destination']
        dest_lng = row['lat_destination']
        
        if row['movement_type'] == 'wait':
            # Keep wait as Point geometry
            feature = {
                "type": "Feature",
                "properties": {
                    "vendor": row['movement_type'],
                    "vol": 0,
                    "duration": row['duration'],
                    "trip_id": i
                },
                "geometry": {
                    "type": "Point",
                    "coordinates": [origin_lng, origin_lat, 0]
                }
            }
        else:
            # Get actual road route for rides and empty moves
            road_coordinates = get_road_route_ors(
                origin_lat, origin_lng, dest_lat, dest_lng, 'openrouteservice.org'
            )
            
            if road_coordinates:
                # Add timestamps to road coordinates
                for j, coord in enumerate(road_coordinates):
                    ratio = j / (len(road_coordinates) - 1) if len(road_coordinates) > 1 else 0
                    timestamp = int((start_time + timedelta(seconds=duration_sec * ratio)).timestamp())
                    coord.append(timestamp)  # Add timestamp as 4th element
                
                coordinates = road_coordinates
            else:
                # Fallback to straight line if routing fails
                coordinates = []  # Your existing interpolation code
            
            feature = {
                "type": "Feature",
                "properties": {
                    "vendor": row['movement_type'],
                    "vol": row.get('revenue', 0),
                    "hexagon_origin": row['hexagon_origin'],
                    "hexagon_destination": row['hexagon_destination'],
                    "duration": row['duration'],
                    "cost": row['cost'],
                    "trip_id": i
                },
                "geometry": {
                    "type": "LineString",
                    "coordinates": coordinates
                }
            }
        
        features.append(feature)


# Create GeoJSON FeatureCollection
geojson = {
    "type": "FeatureCollection",
    "features": features
}

# Save to JSON
with open('outputs/trip_layer_data.json', 'w') as f:
    json.dump(geojson, f, indent=2)



In [58]:
import openrouteservice as ors

def get_road_route_ors(origin_lat, origin_lng, dest_lat, dest_lng, api_key):
    client = ors.Client(key=api_key)  # Free API key from openrouteservice.org
    
    coords = [[origin_lng, origin_lat], [dest_lng, dest_lat]]
    
    route = client.directions(
        coordinates=coords,
        profile='driving-car',
        format='geojson'
    )
    
    if route:
        coordinates = route['features'][0]['geometry']['coordinates']
        # Add elevation and timestamp if needed
        return [[coord[0], coord[1], 0] for coord in coordinates]
    return None